Test manager of root functionality

In [1]:

import sys
from pathlib import Path
import seaborn as sns

In [2]:

LCL_PATH  = str(Path().cwd().parent.parent)
ROOT_PATH = str(Path(LCL_PATH).parent.parent)
EXTERN_CMD_PATH = str(Path(ROOT_PATH))
GENERIC_PATH    = str(Path(ROOT_PATH)/"GeneticAlgorithmManager")
SPECIFIC_PATH   = str(Path(ROOT_PATH)/"extension")
print("""
root path:\t{}
local path:\t{}
generic_path:\t{}
project_specific_path:\t{}
extern_cmd_path: \t{}""".format(ROOT_PATH, LCL_PATH, GENERIC_PATH, SPECIFIC_PATH, EXTERN_CMD_PATH))


root path:	/home/gheorghe/Desktop/Proiecte/master/MIN_metode_inspirate_din_natura
local path:	/home/gheorghe/Desktop/Proiecte/master/MIN_metode_inspirate_din_natura/laborator_1/TTP
generic_path:	/home/gheorghe/Desktop/Proiecte/master/MIN_metode_inspirate_din_natura/GeneticAlgorithmManager
project_specific_path:	/home/gheorghe/Desktop/Proiecte/master/MIN_metode_inspirate_din_natura/extension
extern_cmd_path: 	/home/gheorghe/Desktop/Proiecte/master/MIN_metode_inspirate_din_natura


In [3]:

# adding local_folder to the system path
sys.path.append(LCL_PATH)
sys.path.append(ROOT_PATH)
sys.path.append(GENERIC_PATH)
sys.path.append(SPECIFIC_PATH)

from sys_function import * # este in root

In [29]:

sys_remove_modules("imshow")
sys_remove_modules("algoritm_genetic") # facem update doar daca modificam ceva in generic
sys_remove_modules("genoms") # 
sys_remove_modules("builds.init_population.init_population_tabu_search_ttp")
sys_remove_modules("builds.dataset_utils.dataset_ttp_man")
sys_remove_modules("builds.dataset_utils.ttp_generator")

from imshow import *
from algoritm_genetic import *
from genoms import *
from builds.init_population.init_population_tabu_search_ttp import *
from builds.dataset_utils.dataset_ttp_man import *
from builds.dataset_utils.ttp_generator import *

# Data aquisition

In [5]:

ttp_generator = TTPGenerator("{}/data".format(LCL_PATH))
dataset = ttp_generator(nodes_file="NODE_COORD_SECTION.csv",
                        items_file="ITEMS_SECTION.csv")
dataset

{'GENOME_LENGTH': 280,
 'distance': array([[ 0., 20., 24., ..., 43., 34., 18.],
        [20.,  0., 18., ..., 36., 28.,  9.],
        [24., 18.,  0., ..., 20., 11., 10.],
        ...,
        [43., 36., 20., ...,  0.,  9., 29.],
        [34., 28., 11., ...,  9.,  0., 20.],
        [18.,  9., 10., ..., 29., 20.,  0.]], shape=(280, 280)),
 'coords': array([[288, 149],
        [288, 129],
        [270, 133],
        [256, 141],
        [256, 157],
        [246, 157],
        [236, 169],
        [228, 169],
        [228, 161],
        [220, 169],
        [212, 169],
        [204, 169],
        [196, 169],
        [188, 169],
        [196, 161],
        [188, 145],
        [172, 145],
        [164, 145],
        [156, 145],
        [148, 145],
        [140, 145],
        [148, 169],
        [164, 169],
        [172, 169],
        [156, 169],
        [140, 169],
        [132, 169],
        [124, 169],
        [116, 161],
        [104, 153],
        [104, 161],
        [104, 169],
        [ 90

In [33]:
dataset_obj = DatasetTTPMan(dataset)

# InitPopulationTabuSearch

In [34]:

obj = InitPopulationTabuSearch("init", dataset_man=dataset_obj, city=0, window_size=4, v_min=0.1, v_max=1, W=25936, R=5.61)
obj.setParameters(GENOME_LENGTH=dataset["GENOME_LENGTH"])

In [17]:

genoms = obj(10)

In [18]:

for elite_pos in np.arange(2):
    print(elite_pos)
    tsp_individ = genoms["tsp"][elite_pos]
    kp_individ  = genoms["kp"][elite_pos]
    score = dataset_obj.computeIndividScore(tsp_individ, kp_individ, v_min=0.1, v_max=1, W=25936, R=5.61)
    profit = dataset_obj.computeIndividProfit(kp_individ)
    weight = dataset_obj.computeIndividWeight(kp_individ)
    print("score {}, weight {}, profit {}".format(score, weight, profit))
    map_imgp = ttp_generator.putRoutesOnMap(tsp_individ)
    image_show_wait(map_imgp, window_name="Parent")

0
score -14297.85019191177, weight 25749, profit 35249
Key 32
1
score -29682.648441312558, weight 25784, profit 36984
Key 32


In [31]:

#route, city = np.arange(280, dtype=np.int32), 0
is_find = True
count = 0
ret_route = route.copy()
while (is_find and (count < 1000)):
    ret_route, is_find = obj.insertion_tabu_search_distance(ret_route, city)
    old_distance = dataset_obj.computeIndividDistance(route)
    new_distance = dataset_obj.computeIndividDistance(ret_route)
    count += 1
print("old distance {}, new distance {}, count {}".format(old_distance, new_distance, count))
map_imgo = ttp_generator.putRoutesOnMap(route)
map_imgp = ttp_generator.putRoutesOnMap(ret_route)
image_show(map_imgo, window_name="Old")
image_show_wait(map_imgp, window_name="Tabu")

old distance 2808.0, new distance 2806.0, count 2
Key 32


In [ ]:

#tmp_profit = genoms["kp"][0]
for idx in range(10):
    tmp_profit, is_find = obj.tabu_search_score(ret_route, tmp_profit, v_min=0.1, v_max=1, W=25936, R=5.61)
    tmp_profit = obj.erase_weightier_objects(tmp_profit, 25936)
    score  = dataset_obj.computeIndividScore(ret_route, tmp_profit, v_min=0.1, v_max=1, W=25936, R=5.61)
    profit = dataset_obj.computeIndividProfit(tmp_profit)
    weight = dataset_obj.computeIndividWeight(tmp_profit)
    print("is_find {}, score {}, weight {}, profit {}".format(is_find, score, weight, profit))
    